In [1]:
!pip install pin

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 22.6 MB/s eta 0:00:0031m24.5 MB/s eta 0:00:01
  Using cached cmeel-0.53.3-py3-none-any.whl (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.8 MB/s eta 0:00:0031m12.5 MB/s eta 0:00:01
  Using cached cmeel_urdfdom-3.1.1.1-0-py3-none-manylinux_2_28_x86_64.whl (405 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 24.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached cmeel_console_bridge-1.0.2.2-3-py3-none-manylinux_2_28_x86_64.whl (23 kB)
  Using cached cmeel_tinyxml-2.6.2.3-0-py3-none-manylinux_2_28_x86_64.whl (62 kB)
  Using cached cmeel_assimp-5.3.1-0-py3-none-manylinux_2_28_x86_64.whl (14.0 MB)
  Using cached cmeel_octomap-1.9.8.2-3-py3-none-manylinux_2_28_x86_64.whl (696 kB)
  Using cached cmeel_qhull-8.0.2.1-0-py3-none-manylinux_2_28_x86_64.whl (3.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.4 M

In [3]:
!conda install pinocchio -c conda-forge

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0



# All requested packages already installed.



In [4]:
import pinocchio as pin
from pinocchio.utils import *
import numpy as np

model = pin.buildModelFromUrdf('~/home/rshailly/Documents/ur10_urdf.urdf')
data = model.createData()

# Specify joint configuration
q = pin.neutral(model)

# Compute Jacobian at the end-effector
frame_id = model.getFrameId('tool0')  # Adjust as needed
J = pin.computeFrameJacobian(model, data, q, frame_id)

# Compute pseudo-inverse of Jacobian
pseudo_inverse_J = np.linalg.pinv(J)


ModuleNotFoundError: No module named 'pinocchio'

ModuleNotFoundError: No module named 'pin'

In [ ]:
import rclpy
from rclpy.node import Node
from control_msgs.msg import JointTrajectoryControllerState
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from pynput.keyboard import Key, Listener
import pinocchio as pin
from pinocchio.robot_wrapper import RobotWrapper
import numpy as np
import os
from moveit_msgs.srv import GetPositionFK
from std_msgs.msg import Header
from sensor_msgs.msg import JointState
from moveit_msgs.msg import RobotState
from threading import Thread

class UR10Teleop(Node):
    def __init__(self, urdf_path):
        super().__init__('ur10_teleop_keyboard')
        self.publisher = self.create_publisher(JointTrajectory, '/ur10_controller/joint_trajectory', 10)
        self.subscription = self.create_subscription(
            JointTrajectoryControllerState,
            '/ur10_controller/state',
            self.joint_state_callback,
            10)
        self.fk_client = self.create_client(GetPositionFK, '/compute_fk')
        while not self.fk_client.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('/compute_fk service not available, waiting again...')
        
        if not os.path.exists(urdf_path):
            raise ValueError("URDF file not found.")
        self.robot = RobotWrapper.BuildFromURDF(urdf_path)
        
        self.joint_positions = np.zeros(self.robot.model.nq)
        self.joint_names = [name for name in self.robot.model.names if name != "universe"]
        
        listener = Listener(on_press=self.on_press)
        listener.start()
    
    def joint_state_callback(self, msg):
        self.joint_positions = np.array(msg.actual.positions)
    
    def compute_jacobian_and_inverse(self, joint_states):
        self.robot.forwardKinematics(joint_states)
        frame_id = self.robot.model.getFrameId("tool0")
        jacobian = self.robot.computeFrameJacobian(joint_states, frame_id)
        pseudo_inverse_jacobian = np.linalg.pinv(jacobian)
        return jacobian, pseudo_inverse_jacobian
    
    # In your teleoperation script, within the call_fk_service method:
    def call_fk_service(self):
        header = Header()
        header.stamp = self.get_clock().now().to_msg()
        header.frame_id = "base_link"
        fk_link_names = ["tool0"]
        robot_state = RobotState()
        robot_state.joint_state = JointState()
        robot_state.joint_state.name = self.joint_names
        robot_state.joint_state.position = self.joint_positions.tolist()

        req = GetPositionFK.Request()
        req.header = header
        req.fk_link_names = fk_link_names
        req.robot_state = robot_state

        future = self.fk_client.call_async(req)
        try:
            # Wait for the future to complete
            response = future.result()
            self.get_logger().info(f'Result of /compute_fk: {response}')  # Log the response
        except Exception as e:
            self.get_logger().info(f'Service call failed: {e}')

    def on_press(self, key):
        velocity_increment = 0.9
        end_effector_velocity = np.zeros(6)
        
        if key == Key.right:
            end_effector_velocity[1] += velocity_increment
        elif key == Key.left:
            end_effector_velocity[1] -= velocity_increment
        elif key == Key.up:
            end_effector_velocity[0] += velocity_increment
        elif key == Key.down:
            end_effector_velocity[0] -= velocity_increment
        
        jacobian, pseudo_inverse_jacobian = self.compute_jacobian_and_inverse(self.joint_positions)
        
        joint_velocities = np.matmul(pseudo_inverse_jacobian, end_effector_velocity)
        
        time_step = 0.1
        self.joint_positions += joint_velocities * time_step
        print("Tool0 pose: \n", np.matmul(jacobian, self.joint_positions))
        # After updating the joint positions, publish them and then call the FK service in a thread
        self.publish_joint_trajectory(self.joint_positions)
        thread = Thread(target=self.call_fk_service, args=(self))
        thread.start()  # Start the thread

    def publish_joint_trajectory(self, joint_positions):
        msg = JointTrajectory()
        msg.joint_names = self.joint_names
        point = JointTrajectoryPoint()
        point.positions = joint_positions.tolist()
        point.time_from_start.sec = 1  # Adjust as needed
        msg.points.append(point)
        self.publisher.publish(msg)
        self.get_logger().info('Publishing joint trajectory')


def main(args=None):
    rclpy.init(args=args)
    urdf_path = '/home/rshailly/Documents/ur10_description.urdf'
    teleop_node = UR10Teleop(urdf_path)
    try:
        rclpy.spin(teleop_node)
    finally:
    # Ensure all threads are joined here if there are any running
        teleop_node.destroy_node()
        rclpy.shutdown()

if __name__ == '__main__':
    main()


# import rclpy
# from rclpy.node import Node
# from control_msgs.msg import JointTrajectoryControllerState
# from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
# from pynput.keyboard import Key, Listener
# import pinocchio as pin
# from pinocchio.robot_wrapper import RobotWrapper
# import numpy as np
# import os

# class UR10Teleop(Node):
#     def __init__(self, urdf_path):
#         super().__init__('ur10_teleop_keyboard')
#         self.publisher = self.create_publisher(JointTrajectory, '/ur10_controller/joint_trajectory', 10)
#         self.subscription = self.create_subscription(
#             JointTrajectoryControllerState,
#             '/ur10_controller/state',
#             self.joint_state_callback,
#             10)
        
#         # Load the robot model
#         if not os.path.exists(urdf_path):
#             raise ValueError("URDF file not found.")
#         self.robot = RobotWrapper.BuildFromURDF(urdf_path)
        
#         # Initialize joint positions and names
#         self.joint_positions = np.zeros(self.robot.model.nq)
#         self.joint_names = [name for name in self.robot.model.names if name != "universe"]
        
#         # Keyboard listener
#         listener = Listener(on_press=self.on_press)
#         listener.start()
    
#     def joint_state_callback(self, msg):
#         self.joint_positions = np.array(msg.actual.positions)
    
#     def compute_jacobian_and_inverse(self, joint_states):
#         self.robot.forwardKinematics(joint_states)
#         frame_id = self.robot.model.getFrameId("tool0")
#         jacobian = self.robot.frameJacobian(joint_states, frame_id)
#         pseudo_inverse_jacobian = np.linalg.pinv(jacobian)
#         return jacobian, pseudo_inverse_jacobian

#     def on_press(self, key):
#         # Define velocity increments for x and y directions
#         velocity_increment = 0.9  # Adjust as needed for your application
        
#         # Initialize the end effector velocity vector (6D: linear and angular velocities)
#         # We're only interested in linear velocities here, so angular velocities are set to zero.
#         end_effector_velocity = np.zeros(6)
        
#         # Update the velocity based on the key pressed
#         if key == Key.right:
#             end_effector_velocity[1] += velocity_increment  # Increase x velocity
#         elif key == Key.left:
#             end_effector_velocity[1] -= velocity_increment  # Decrease x velocity
#         elif key == Key.up:
#             end_effector_velocity[0] += velocity_increment  # Increase y velocity
#         elif key == Key.down:
#             end_effector_velocity[0] -= velocity_increment  # Decrease y velocity
        
#         # Compute the Jacobian and its pseudo-inverse
#         jacobian, pseudo_inverse_jacobian = self.compute_jacobian_and_inverse(self.joint_positions)
        
#         # Compute the joint velocities by multiplying the pseudo-inverse of the Jacobian with the end effector velocity
#         joint_velocities = np.matmul(pseudo_inverse_jacobian, end_effector_velocity)
#         print("End effector velocities: \n", np.matmul(jacobian,joint_velocities))

        
#         # Update the joint positions by adding a small increment based on the computed joint velocities
#         # This increment is proportional to the joint velocities and assumes a small time step.
#         # Adjust the time step as needed for your application.
#         time_step = 0.1  # Time step for updating positions
#         self.joint_positions += joint_velocities * time_step
        
#         # Publish the updated joint positions as a trajectory
#         self.publish_joint_trajectory(self.joint_positions)


#     def publish_joint_trajectory(self, joint_positions):
#         msg = JointTrajectory()
#         msg.joint_names = self.joint_names
#         point = JointTrajectoryPoint()
#         point.positions = joint_positions.tolist()
#         point.time_from_start.sec = 1  # Adjust as needed
#         msg.points.append(point)
#         self.publisher.publish(msg)
#         self.get_logger().info('Publishing joint trajectory')

# def main(args=None):
#     rclpy.init(args=args)
#     urdf_path = '/home/rshailly/Documents/ur10_description.urdf'
#     teleop_node = UR10Teleop(urdf_path)
#     try:
#         rclpy.spin(teleop_node)
#     finally:
#         teleop_node.destroy_node()
#         rclpy.shutdown()

# if __name__ == '__main__':
#     main()


# import rclpy
# from rclpy.node import Node
# from control_msgs.msg import JointTrajectoryControllerState
# from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
# from pynput.keyboard import Key, Listener
# from pinocchio.robot_wrapper import RobotWrapper
# from std_msgs.msg import Header
# from sensor_msgs.msg import JointState
# # from moveit_msgs.msg import RobotState, GetPositionFK, PositionFKRequest
# import numpy as np
# import os

# class UR10Teleop(Node):
#     def __init__(self, urdf_path):
#         super().__init__('ur10_teleop_keyboard')
#         self.publisher = self.create_publisher(JointTrajectory, '/ur10_controller/joint_trajectory', 10)
#         self.subscription = self.create_subscription(
#             JointTrajectoryControllerState,
#             '/ur10_controller/state',
#             self.joint_state_callback,
#             10)
        
#         # FK Client setup
#         self.fk_client = self.create_client(GetPositionFK, '/compute_fk')
#         while not self.fk_client.wait_for_service(timeout_sec=1.0):
#             self.get_logger().info('/compute_fk service not available, waiting again...')
        
        
#         # Load the robot model
#         if not os.path.exists(urdf_path):
#             raise ValueError("URDF file not found.")
#         self.robot = RobotWrapper.BuildFromURDF(urdf_path)
        
#         # Initialize joint positions and names
#         self.joint_positions = np.zeros(self.robot.model.nq)
#         self.joint_names = [name for name in self.robot.model.names if name != "universe"]
        
#         # Keyboard listener
#         listener = Listener(on_press=self.on_press)
#         listener.start()
    
#     def joint_state_callback(self, msg):
#         self.joint_positions = np.array(msg.actual.positions)
    
#     def compute_jacobian_and_inverse(self, joint_states):
#         self.robot.forwardKinematics(joint_states)
#         frame_id = self.robot.model.getFrameId("tool0")
#         jacobian = self.robot.frameJacobian(joint_states, frame_id)
#         pseudo_inverse_jacobian = np.linalg.pinv(jacobian)
#         return jacobian, pseudo_inverse_jacobian

#     def on_press(self, key):
#         # Define velocity increments for x and y directions
#         velocity_increment = 0.9  # Adjust as needed for your application
        
#         # Initialize the end effector velocity vector (6D: linear and angular velocities)
#         # We're only interested in linear velocities here, so angular velocities are set to zero.
#         end_effector_velocity = np.zeros(6)
        
#         # Update the velocity based on the key pressed
#         if key == Key.right:
#             end_effector_velocity[1] += velocity_increment  # Increase x velocity
#         elif key == Key.left:
#             end_effector_velocity[1] -= velocity_increment  # Decrease x velocity
#         elif key == Key.up:
#             end_effector_velocity[0] += velocity_increment  # Increase y velocity
#         elif key == Key.down:
#             end_effector_velocity[0] -= velocity_increment  # Decrease y velocity
        
#         # Compute the Jacobian and its pseudo-inverse
#         jacobian, pseudo_inverse_jacobian = self.compute_jacobian_and_inverse(self.joint_positions)
        
#         # Compute the joint velocities by multiplying the pseudo-inverse of the Jacobian with the end effector velocity
#         joint_velocities = np.matmul(pseudo_inverse_jacobian, end_effector_velocity)
#         print("End effector velocities: \n", np.matmul(jacobian,joint_velocities))

        
#         # Update the joint positions by adding a small increment based on the computed joint velocities
#         # This increment is proportional to the joint velocities and assumes a small time step.
#         # Adjust the time step as needed for your application.
#         time_step = 0.1  # Time step for updating positions
#         self.joint_positions += joint_velocities * time_step
        
#         # Publish the updated joint positions as a trajectory
#         self.publish_joint_trajectory(self.joint_positions)

#         # After updating the joint positions, compute the pose of 'tool0'
#         self.compute_and_print_tool0_pose()

#     def compute_fk_and_print_tool0_pose(self):
#         header = Header()
#         header.stamp = self.get_clock().now().to_msg()
#         header.frame_id = "base_link"

#         fk_link_names = ["tool0"]

#         robot_state = RobotState()
#         robot_state.joint_state = JointState()
#         robot_state.joint_state.name = self.joint_names
#         robot_state.joint_state.position = self.joint_positions.tolist()

#         # Create and send the FK request
#         req = GetPositionFK.Request()
#         req.header = header
#         req.fk_link_names = fk_link_names
#         req.robot_state = robot_state
#         future = self.fk_client.call_async(req)

#         def get_fk_response(future):
#             try:
#                 response = future.result()
#                 if response.error_code.val == response.error_code.SUCCESS:
#                     for transform in response.pose_stamped:
#                         self.get_logger().info(f"Pose of {transform.header.frame_id}: {transform.pose}")
#                 else:
#                     self.get_logger().error('Failed to compute FK')
#             except Exception as e:
#                 self.get_logger().error(f'Service call failed {e}')

#         future.add_done_callback(get_fk_response)


#     def publish_joint_trajectory(self, joint_positions):
#         msg = JointTrajectory()
#         msg.joint_names = self.joint_names
#         point = JointTrajectoryPoint()
#         point.positions = joint_positions.tolist()
#         point.time_from_start.sec = 1  # Adjust as needed
#         msg.points.append(point)
#         self.publisher.publish(msg)
#         self.get_logger().info('Publishing joint trajectory')

# def main(args=None):
#     rclpy.init(args=args)
#     urdf_path = '/home/rshailly/Documents/ur10_description.urdf'
#     teleop_node = UR10Teleop(urdf_path)
#     try:
#         rclpy.spin(teleop_node)
#     finally:
#         teleop_node.destroy_node()
#         rclpy.shutdown()

# if __name__ == '__main__':
#     main()




# import rclpy
# from rclpy.node import Node
# from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
# from builtin_interfaces.msg import Duration
# from pynput.keyboard import Key, Listener
# from urdf_parser_py.urdf import URDF

# class UR10Teleop(Node):
#     def __init__(self):
#         super().__init__('ur10_teleop_keyboard')
#         self.publisher = self.create_publisher(JointTrajectory, '/ur10_controller/joint_trajectory', 10)
#         self.joint_names = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
#                             'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
#         # Initialize with a neutral position or your robot's home position
        
#         self.positions = [0.0, -1.57, 1.57, 0.0, 0.0, 0.0] ## TODO: We need to get this from /ur10_controller/state -> actual -> positions
        
#         # Adjust these increments for suitable movement sensitivity
#         self.xy_increment = 0.1
#         self.z_increment = 0.1

#         # Start listening to keyboard inputs
#         listener = Listener(on_press=self.on_press)
#         listener.start()

#     def send_command(self):
#         msg = JointTrajectory()
#         msg.joint_names = self.joint_names
#         point = JointTrajectoryPoint()
#         point.positions = self.positions
#         point.time_from_start = Duration(sec=1)  # Adjust time for movement completion
#         msg.points.append(point)

#         self.publisher.publish(msg)
#         self.get_logger().info('Publishing joint trajectory')

#     def on_press(self, key):
#         if key == Key.up:
#             self.positions[1] += self.xy_increment  # Adjust Y joint
#         elif key == Key.down:
#             self.positions[1] -= self.xy_increment  # Adjust Y joint
#         elif key == Key.right:
#             self.positions[0] += self.xy_increment  # Adjust X joint
#         elif key == Key.left:
#             self.positions[0] -= self.xy_increment  # Adjust X joint
#         elif key == 'w':
#             self.positions[2] += self.z_increment  # Adjust Z joint
#         elif key == 'x':
#             self.positions[2] -= self.z_increment  # Adjust Z joint
#         self.send_command()

# def main(args=None):
#     rclpy.init(args=args)
#     teleop = UR10Teleop()
#     try:
#         rclpy.spin(teleop)
#     except KeyboardInterrupt:
#         pass
#     finally:
#         teleop.destroy_node()
#         rclpy.shutdown()

# if __name__ == '__main__':
#     main()


In [ ]:


import pinocchio as pin
from pinocchio.robot_wrapper import RobotWrapper
import numpy as np
import os

class RobotArm:
    def __init__(self, urdf_path):
        # Ensure the URDF file is found at the specified path
        if not os.path.exists(urdf_path):
            raise ValueError("URDF file not found at specified path.")
        self.robot = RobotWrapper.BuildFromURDF(urdf_path)
        self.model = self.robot.model
        self.data = self.robot.data

    def compute_jacobian(self, joint_states, frame_name="tool0"):
       
        if len(joint_states) != self.robot.model.nq:
            raise ValueError("Joint state vector does not match the model's number of joints.")
        self.robot.forwardKinematics(joint_states)
        frame_id = self.robot.model.getFrameId(frame_name)
        jacobian = self.robot.frameJacobian(joint_states, frame_id)
        return jacobian

    def pseudo_inverse(self, matrix, tolerance=1e-4):
        u, s, vh = np.linalg.svd(matrix, full_matrices=False)
        s_inv = np.array([0 if abs(x) < tolerance else 1/x for x in s])
        return np.dot(vh.T, np.dot(np.diag(s_inv), u.T))
    
    def compute_fk(self, q):
        


# Example usage
if __name__ == "__main__":
    # print(pin.Model())
    urdf_path = '/home/rshailly/Documents/ur10_description.urdf'
    robot_arm = RobotArm(urdf_path)
    
    # Example joint state; replace with actual joint states as needed
    
    joint_states = np.array([3.080001785198407, -2.3760839802651876,  -1.135113218857268, -1.214480224517489, 1.390300467660687, 1.3903042907211018] )
    q_dot = 0.01*joint_states
    x_dot = np.array([-0.11674441,  0.39035987,  0.27991097, -0.20627489,  0.37872736, -0.2487736])
    jacobian = robot_arm.compute_jacobian(joint_states)
    print("Jacobian:\n", jacobian)
    
    pseudo_inverse_jacobian = robot_arm.pseudo_inverse(jacobian)
    print("Pseudo-inverse of the Jacobian:\n", pseudo_inverse_jacobian)
    print("\n\n")
    print("End effector velocity:", np.matmul(jacobian,0.1*joint_states))
    print("\n\n")
    print("Joint Velocities:", np.matmul(pseudo_inverse_jacobian,x_dot))

In [1]:
a = [0,1,2,3,4,5]

In [2]:
a[3:]

[3, 4, 5]